# Poker Agent - SARSA

In [1]:
import pandas as pd
import numpy as np
import time
import itertools as it

In [27]:
Q = pd.read_csv(r"C:\Users\cnaka\Class Notes\32020 Reinforcement Learning\Q.csv")
Q = Q.drop("Unnamed: 0",axis=1)
Q = pd.DataFrame(Q,columns = ["bet","raise","check","fold","state"])

In [28]:
Q = np.zeros((722,4))
a = [True,False]
indx = []
for i in range(1,14):
    for j in range(1,14):
        for k in a:
            temp = [i,j]
            temp = sorted(temp[0:2])+[k]
            indx.append(tuple(temp))
indx2=[]
[indx2.append(i) for i in indx if i not in indx2 ]
indx = []
a = ["flop","turn","river"]
for i in range(0,9):
    for j in range(0,2):
        for k in range(0,2):
            for z in range(0,9):
                for l in a:
                    if (z<=i):
                        indx.append(tuple([i,j,k,z,l]))
indx.extend(indx2)
Q = pd.DataFrame(Q,columns = ["bet","raise","check","fold"])
Q["state"]=indx


In [29]:
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='river')&(Q.loc[i,'state'][0]>2))
    except:
        a.append(False)
Q.loc[a,'bet'] = 4
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='river')&(Q.loc[i,'state'][0] in [range(1,3)]))
    except:
        a.append(False)
Q.loc[a,'bet'] = 1.75
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='turn')&(Q.loc[i,'state'][0]>2))
    except:
        a.append(False)
Q.loc[a,'bet'] = 3
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='turn')&(Q.loc[i,'state'][0] == 2))
    except:
        a.append(False)
Q.loc[a,'bet'] = 1.75
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='turn')&(Q.loc[i,'state'][0] == 1))
    except:
        a.append(False)
Q.loc[a,'bet'] = 1.25
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='flop')&(Q.loc[i,'state'][0]>2))
    except:
        a.append(False)
Q.loc[a,'bet'] = 2
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='flop')&(Q.loc[i,'state'][0] ==2))
    except:
        a.append(False)
Q.loc[a,'bet'] = 1.75
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='flop')&(Q.loc[i,'state'][0] == 1))
    except:
        a.append(False)
Q.loc[a,'bet'] = 1.25
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='river')&(Q.loc[i,'state'][0]>2))
    except:
        a.append(False)
Q.loc[a,'raise'] = 5
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='river')&(Q.loc[i,'state'][0] in [range(1,3)]))
    except:
        a.append(False)
Q.loc[a,'raise'] = 1.5
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='turn')&(Q.loc[i,'state'][0]>2))
    except:
        a.append(False)
Q.loc[a,'raise'] = 4
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='turn')&(Q.loc[i,'state'][0] == 2))
    except:
        a.append(False)
Q.loc[a,'raise'] = 3
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='turn')&(Q.loc[i,'state'][0] == 1))
    except:
        a.append(False)
Q.loc[a,'raise'] = 1.3
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='flop')&(Q.loc[i,'state'][0]>2))
    except:
        a.append(False)
Q.loc[a,'raise'] = 2
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='flop')&(Q.loc[i,'state'][0] ==2))
    except:
        a.append(False)
Q.loc[a,'raise'] = 1.75
a = []
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][3]=='flop')&(Q.loc[i,'state'][0] == 1))
    except:
        a.append(False)
Q.loc[a,'raise'] = 1.25
a=[]
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][0] == 0))
    except:
        a.append(False)
Q.loc[a,'raise'] = -1
a=[]
for i in range(0,len(Q['state'])):
    try:
        a.append((Q.loc[i,'state'][0] == 0))
    except:
        a.append(False)
Q.loc[a,'bet'] = -.5

## Create Poker Environment

In [33]:
def deal():
    deck= list(range(1,53))
    hand_p1 = list(np.random.choice(deck,size=2,replace=False))
    deck = [i for i in deck if i not in hand_p1]
    hand_p1.sort()
    hand_p2 = list(np.random.choice(deck,size=2,replace=False))
    deck = [i for i in deck if i not in hand_p2]
    hand_p2.sort()
    return hand_p1,hand_p2,deck

In [34]:
def flop(deck):
    flop=list(np.random.choice(deck,size=3,replace=False))
    deck = [i for i in deck if i not in flop]
    return flop,deck

In [35]:
def turn(board,deck):
    turn=list(np.random.choice(deck,size=1,replace=False))
    board.extend(turn)
    deck = [i for i in deck if i not in turn]
    return board,deck

In [36]:
def river(board,deck):
    river=list(np.random.choice(deck,size=1,replace=False))
    board.extend(river)
    deck = [i for i in deck if i not in river]
    return board,deck

In [8]:
def check_straight(rank):
    straight, high_card = False, -1
    if all([j in rank for j in [0,1,2,3,12]]):
        straight, high_card = True, 3
    for i in range(4, 13):
        if all([j in rank for j in [x for x in range(i-4, i+1)]]):
            straight, high_card = True, i
    return straight, high_card

In [9]:
def hand_strength(hand,board):
    #0 is a 2, 1 is a 3, ..., 9 is Jack, 10 is Queen, 11 is King, 12 is Ace
    #suits are 0 through 3
    handfull=hand+board
    best_hand = [0,0,0,0,0]
    hand_strength = 0
    flushdraw=0
    straightdraw=0
    if (len(handfull)<5):
        tiebreak = [0,0,0]
        if (np.bincount([x//13 for x in handfull]).max() == 4):
            if np.bincount([x//13 for x in board]).max() == 4:
                flushdraw = 0
            else:
                flushdraw = 1
        for i in range(1,14):
            a = [x%13 for x in handfull]
            a.append(i)
            if ((check_straight(a)[0]== True)&(check_straight(hand)[0]==False)):
                straightdraw = 1
                break
            else:
                straight_draw = 0
        for i in it.combinations(handfull,len(handfull)):
            rank = [x%13 for x in i]
            suit = [x//13 for x in i]
            p1_strength =0
    
            #look for straights
            p1_straight, p1_high_card_straight = check_straight(rank)
    
            p1_flush = np.bincount(suit).max()>=5
            p1_max_freq = np.bincount(rank).max()
    
            p1_four = p1_max_freq == 4
            p1_three = p1_max_freq == 3
            p1_pair = p1_max_freq == 2
            rank_unique = list(set(rank))
            rank_unique.sort()
            p1_two_pair = [rank.count(x) for x in rank_unique].count(2)>=2
   
            p1_full_house = ([rank.count(x) for x in rank_unique].count(3) and [rank.count(x) for x in rank_unique].count(2)) or [rank.count(x) for x in rank_unique].count(3) == 2
            #check for straight flush
            if p1_flush and p1_straight:
                p1_flush_suit = np.bincount(suit).argmax()
                p1_flush_cards = [rank[i] for i, x in enumerate(suit) if x==p1_flush_suit]
                p1_straight_flush, p1_straight_flush_high_card = check_straight(p1_flush_cards)
            else:
                p1_straight_flush = False
    
            if p1_straight_flush:
                p1_strength = 8
            elif p1_four:
                p1_strength = 7

            elif p1_full_house:
                p1_strength = 6

            elif p1_flush:
                p1_strength = 5
            elif p1_straight:
                p1_strength = 4
            elif p1_three:
                p1_strength = 3

            elif p1_two_pair:
                p1_strength = 2

            elif p1_pair:
                p1_strength = 1

            else:
                p1_strength = 0
            #determine if curent hand is stronger than current best
            if (p1_strength>hand_strength):
                best_hand = i
                hand_strength = p1_strength
        return best_hand,hand_strength,flushdraw,straightdraw,tiebreak
    
        
    else:
        if (np.bincount([x//13 for x in handfull]).max() == 4):
            if np.bincount([x//13 for x in board]).max() == 4:
                flushdraw = 0
            else:
                flushdraw = 1
        for i in range(1,14):
            a = [x%13 for x in handfull]
            a.append(i)
            if ((check_straight(a)[0]== True)&(check_straight(hand)[0]==False)):
                straightdraw = 1
                break
            else:
                straight_draw = 0
        for i in it.combinations(handfull,5):
            rank = [x%13 for x in i]
            suit = [x//13 for x in i]
            p1_strength =0
    
            #look for straights
            p1_straight, p1_high_card_straight = check_straight(rank)
    
            p1_flush = np.bincount(suit).max()>=5
            p1_max_freq = np.bincount(rank).max()
    
            p1_four = p1_max_freq == 4
            p1_three = p1_max_freq == 3
            p1_pair = p1_max_freq == 2
            rank_unique = list(set(rank))
            rank_unique.sort()
            p1_two_pair = [rank.count(x) for x in rank_unique].count(2)>=2
   
            p1_full_house = ([rank.count(x) for x in rank_unique].count(3) and [rank.count(x) for x in rank_unique].count(2)) or [rank.count(x) for x in rank_unique].count(3) == 2
            #check for straight flush
            if p1_flush and p1_straight:
                p1_flush_suit = np.bincount(suit).argmax()
                p1_flush_cards = [rank[i] for i, x in enumerate(suit) if x==p1_flush_suit]
                p1_straight_flush, p1_straight_flush_high_card = check_straight(p1_flush_cards)
            else:
                p1_straight_flush = False
    
            if p1_straight_flush:
                p1_strength = 8
            elif p1_four:
                p1_strength = 7
                p1_quads = np.bincount(rank).argmax()
                kicker = rank_unique[-1]
                if kicker == p1_quads:
                    kicker = rank_unique[-2]
            elif p1_full_house:
                p1_strength = 6
                threes = [x for x in rank_unique if rank.count(x)==3]
                twos = [x for x in rank_unique if rank.count(x)==2]
            elif p1_flush:
                p1_strength = 5
            elif p1_straight:
                p1_strength = 4
            elif p1_three:
                p1_strength = 3
                trips = np.bincount(rank).argmax()
                kicker1 = [x for x in rank_unique if x!=trips][-1]
                kicker2 = [x for x in rank_unique if x!=trips][-2]
            elif p1_two_pair:
                p1_strength = 2
                pairs = [x for x in rank_unique if rank.count(x)==2]
                pairs.sort()
                kicker = [x for x in rank_unique if x not in pairs][-1]
            elif p1_pair:
                p1_strength = 1
                pair = [x for x in rank_unique if rank.count(x)==2]
                pair = pair[0]
                kicker = [x for x in rank if x != pair]
                kicker.sort()
                kicker1 = kicker[2]
                kicker2 = kicker[1]
                kicker3 = kicker[0]
            else:
                p1_strength = 0
            #determine if curent hand is stronger than current best
            if (p1_strength>hand_strength):
                best_hand = i
                hand_strength = p1_strength
                if (p1_strength == 8):
                    tiebreak=p1_straight_flush_high_card
                elif (p1_strength == 7):
                    tiebreak= kicker
                elif (p1_strength ==6):
                    tiebreak=[threes,twos]
                elif (p1_strength == 5):
                    tiebreak=i
                elif (p1_strength == 4):
                    tiebreak=i[-1]
                elif (p1_strength == 3):
                    tiebreak = [trips,kicker1,kicker2]
                elif (p1_strength ==2):
                    tiebreak = [pairs[1],pairs[0],kicker]
                elif (p1_strength ==1):
                    tiebreak = [pair,kicker1,kicker2,kicker3]
                elif (p1_strength ==0):
                    tiebreak =i
            
            elif (p1_strength==hand_strength):
                if (p1_strength == 8):
                    if (p1_straight_flush_high_card>tiebreak):
                        best_hand = i
                        tiebreak=p1_straight_flush_high_card
                elif (p1_strength == 7):
                    if (kicker>tiebreak):
                        best_hand=i
                        tiebreak = kicker
                elif (p1_strength == 6):
                    if ((threes>tiebreak[0])|((threes==tiebreak[0]) & (twos > tiebreak[1]))):
                        best_hand=i
                        tiebreak = [threes,twos]
                elif (p1_strength == 5):
                    if(max(i)>max(best_hand)):
                        best_hand = i
                        tiebreak = i
                elif (p1_strength == 4):
                    if (max(i)>max(best_hand)):
                        best_hand = i
                elif (p1_strength == 3):
                    if ((trips>tiebreak[0])|((trips==tiebreak[0])&(kicker1>tiebreak[1]))|((trips==tiebreak[0])&(kicker1==tiebreak[1])&(kicker2>tiebreak[2]))):
                        best_hand=i
                        tiebreak = [trips,kicker1,kicker2]
                elif (p1_strength ==2):
                    if ((pairs[1]>tiebreak[0])|((pairs[1]==tiebreak[0])&(pairs[0]>tiebreak[1]))|((pairs[1]==tiebreak[0])&(pairs[0]==tiebreak[1])&(kicker>tiebreak[2]))):
                        best_hand = i
                        tiebreak = [pairs[1],pairs[0],kicker]
                elif (p1_strength ==1):
                    if((pair>tiebreak[0])|((pair == tiebreak[0])&(kicker1>tiebreak[1]))|((pair == tiebreak[0])&(kicker1==tiebreak[1])&(kicker2>tiebreak[2]))|((pair == tiebreak[0])&(kicker1==tiebreak[1])&(kicker2==tiebreak[2])&(kicker3>tiebreak[3]))):
                        best_hand = i
                        tiebreak = [pair,kicker1,kicker2,kicker3]
                elif (p1_strength == 0):
                    if (max(i)>max(best_hand)):
                        best_hand = i
                        tiebreak=i
        return best_hand,hand_strength,flushdraw,straightdraw,tiebreak       



In [10]:
def find_winner(p1,p2,board):
    p1strength=hand_strength(p1,board)
    p2strength=hand_strength(p2,board)
    result = "None"
    if (p1strength[1]>p2strength[1]):
        return "p1"
    elif (p2strength[1]>p1strength[1]):
        return "p2"
    else:
        if (p1strength[1]==8):
            if (p1strength[4]>p2strength[4]):
                return "p1"
            elif (p2strength[4]>p1strength[4]):
                return "p2"
            else:
                return "split"
        elif (p1strength[1]==7):
            if (p1strength[4]>p2strength[4]):
                return "p1"
            elif (p2strength[4]>p1strength[4]):
                return "p2"
            else:
                return "split"
        elif (p1strength[1]==6):
            if (p1strength[4][0]>p2strength[4][0]):
                return "p1"
            elif (p2strength[4][0]>p1strength[4][0]):
                return "p2"
            elif (p1strength[4][1]>p2strength[4][1]):
                return "p1"
            elif (p2strength[4][1]>p1strength[4][1]):
                return "p2"
            else:
                return "split"
        elif (p1strength[1]==5):           
            if (p1strength[0][4]>p2strength[0][4]):
                return "p1"
            elif (p2strength[0][4]>p1strength[0][4]):
                return "p2"
            elif (p1strength[0][3]>p2strength[0][3]):
                return "p1"
            elif (p2strength[0][3]>p1strength[0][3]):
                return "p2"
            elif (p1strength[0][2]>p2strength[0][2]):
                return "p1"
            elif (p2strength[0][2]>p1strength[0][2]):
                return "p2"
            elif (p1strength[0][1]>p2strength[0][1]):
                return "p1"
            elif (p2strength[0][1]>p1strength[0][1]):
                return "p2"
            elif (p1strength[0][0]>p2strength[0][0]):
                return "p1"
            elif (p2strength[0][0]>p1strength[0][0]):
                return "p2"
            else:
                return "split"
        
        elif (p1strength[1]==4):           
            if (max(p1strength[0])>max(p2strength[0])):
                return "p1"
            elif (max(p2strength[0])>max(p1strength[0])):
                return "p2"
            else:
                return "split"
        elif (p1strength[1]==3):
            if (p1strength[4][0]>p2strength[4][0]):
                return "p1"
            elif (p2strength[4][0]>p1strength[4][0]):
                return "p2"
            elif (p1strength[4][1]>p2strength[4][1]):
                return "p1"
            elif (p2strength[4][1]>p1strength[4][1]):
                return "p2"
            elif (p1strength[4][2]>p2strength[4][2]):
                return "p1"
            elif (p2strength[4][2]>p1strength[4][2]):
                return "p2"
            else:
                return "split"
        elif (p1strength[1]==2):
            if (p1strength[4][0]>p2strength[4][0]):
                return "p1"
            elif (p2strength[4][0]>p1strength[4][0]):
                return "p2"
            elif (p1strength[4][1]>p2strength[4][1]):
                return "p1"
            elif (p2strength[4][1]>p1strength[4][1]):
                return "p2"
            elif (p1strength[4][2]>p2strength[4][2]):
                return "p1"
            elif (p2strength[4][2]>p1strength[4][2]):
                return "p2"
            else:
                return "split"
        elif (p1strength[1]==1):
            if (p1strength[4][0]>p2strength[4][0]):
                return "p1"
            elif (p2strength[4][0]>p1strength[4][0]):
                return "p2"
            elif (p1strength[4][1]>p2strength[4][1]):
                return "p1"
            elif (p2strength[4][1]>p1strength[4][1]):
                return "p2"
            elif (p1strength[4][2]>p2strength[4][2]):
                return "p1"
            elif (p2strength[4][2]>p1strength[4][2]):
                return "p2"
            elif (p1strength[4][3]>p2strength[4][3]):
                return "p1"
            elif (p2strength[4][3]>p1strength[4][3]):
                return "p2"
            else:
                return "split"
        elif (p1strength[1]==0):
            if (p1strength[0][4]>p2strength[0][4]):
                return "p1"
            elif (p2strength[0][4]>p1strength[0][4]):
                return "p2"
            elif (p1strength[0][3]>p2strength[0][3]):
                return "p1"
            elif (p2strength[0][3]>p1strength[0][3]):
                return "p2"
            elif (p1strength[0][2]>p2strength[0][2]):
                return "p1"
            elif (p2strength[0][2]>p1strength[0][2]):
                return "p2"
            elif (p1strength[0][1]>p2strength[0][1]):
                return "p1"
            elif (p2strength[0][1]>p1strength[0][1]):
                return "p2"
            elif (p1strength[0][0]>p2strength[0][0]):
                return "p1"
            elif (p2strength[0][0]>p1strength[0][0]):
                return "p2"
            else:
                return "split"
        

In [11]:
def preflopstate(hand):
    state = [hand[0]%13+1,hand[1]%13+1]
    state.sort()
    state.append(hand[0]//13==hand[1]//13)
    return tuple(state)

In [12]:
def state(hand,board,stage):
    state = list(hand_strength(hand,board)[1:4])
    state.append(hand_strength([],board)[1])
    state.append(stage)
    return tuple(state)

In [30]:
def decision(state,availablemoves):
    d0 = "None"
    if np.random.uniform(0,1,1)>=.1:
        d0 = Q[Q["state"]==state].loc[:,availablemoves].idxmax(axis=1).iloc[0]
    else:
        d0 = np.random.choice(availablemoves,size =1)[0]
    return d0

## Perform Agent Training

In [15]:
def updateQ(Q,state1,state2,action1,action2,reward,gamma):
    if (state2 == None):
        #if (state1[0:3] == (0,0,0)):
            #print(state1,state2,action1,action2,reward,gamma,gamma*(reward-float(Q.loc[Q["state"]==state1,action1])))
        Q.loc[Q["state"]==state1,action1]= float(Q.loc[Q["state"]==state1,action1])+gamma*(reward-float(Q.loc[Q["state"]==state1,action1]))
    else:
        #if (state1[0:3] == (0,0,0)):
            #print(state1,state2,action1,action2,reward,gamma,float(Q.loc[Q['state']==state2,action2]),float(Q.loc[Q["state"]==state1,action1]),gamma*(reward+float(Q.loc[Q['state']==state2,action2])-float(Q.loc[Q["state"]==state1,action1])))
        Q.loc[Q["state"]==state1,action1]= float(Q.loc[Q["state"]==state1,action1])+gamma*(reward+float(Q.loc[Q['state']==state2,action2])-float(Q.loc[Q["state"]==state1,action1]))
    return Q

In [16]:
def simgame(Q,gamma):
    betsize = 1
    ante = .25
    p1,p2,deck =deal()
    #firstmove=np.random.choice(["p1","p2"],1)[0]
    pot = 2*ante
    #####preflop betting
    rewardp1 = -ante
    rewardp2 = -ante
    statep1 = preflopstate(p1)
    statep2 = preflopstate(p2)
    actionp1 = decision(statep1,["bet","check"])
    if (actionp1 == "bet"):
        pot = pot+betsize
        rewardp1 = -betsize-ante
        actionp2 = decision(statep2,["bet","fold","raise"])
        if (actionp2 == "bet"):
            pot = pot+betsize
            rewardp2 = -betsize-ante
        elif (actionp2 == "raise"):
            betsize = betsize + 1
            pot = pot+betsize
            rewardp2 = -betsize-ante
            while (betsize <5):
                action1p1 = decision(statep1,["bet","raise","fold"])
                Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
                actionp1 = action1p1
                if (actionp1 == "bet"):
                    pot=pot+betsize
                    rewardp1 = -betsize
                    betsize = 1
                    break
                elif (actionp1 == "raise"):
                    betsize = betsize+1
                    pot = pot+betsize
                    rewardp1 = -betsize
                    action1p2=decision(statep2,["bet","raise","fold"])
                    Q = updateQ(Q,statep2,statep2,actionp2,action1p2,rewardp2,gamma)
                    actionp2 = action1p2
                    if (actionp2 == "bet"):
                        pot=pot+betsize
                        rewardp1 = -betsize
                        betsize = 1
                        break
                    elif (actionp2=='raise'):
                        betsize = betsize+1
                        pot = pot+betsize
                        rewardp2=-betsize
                    elif (actionp2 == 'fold'):
                        Q = updateQ(Q,statep2,None,actionp2,None,rewardp2,gamma)
                        Q = updateQ(Q,statep1,None,actionp1,None,pot+rewardp1,gamma)
                        return Q
                elif (actionp1 == 'fold'):
                    Q = updateQ(Q,statep1,None,actionp1,None,rewardp1,gamma)
                    Q = updateQ(Q,statep2,None,actionp2,None,pot+rewardp2,gamma)
                    return Q
        elif (actionp2 == "fold"):
            Q = updateQ(Q,statep2,None,"fold",None,rewardp2,gamma)
            Q = updateQ(Q,statep1,None,"bet",None,pot+rewardp1,gamma)
            return Q
    elif (actionp1 == "check"):
        actionp2 = decision(statep2,["bet","check"])
        if (actionp2 == "bet"):
            pot = pot+betsize
            rewardp2=-ante-betsize
            action1p1=decision(statep1,["bet","fold","raise"])
            Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
            actionp1 = action1p1
            if (actionp1 == "bet"):
                pot=pot+betsize
                rewardp1 = -betsize
            elif (actionp1 == "raise"):
                betsize = betsize + 1
                pot = pot+betsize
                rewardp1 = -betsize-ante
                while (betsize <5):
                    action1p2 = decision(statep2,["bet","raise","fold"])
                    Q = updateQ(Q,statep2,statep2,actionp2,action1p2,rewardp2,gamma)
                    actionp2 = action1p2
                    if (actionp2 == "bet"):
                        pot=pot+betsize
                        rewardp2 = -betsize
                        betsize = 1
                        break
                    elif (actionp2 == "raise"):
                        betsize = betsize+1
                        pot = pot+betsize
                        rewardp2 = -betsize
                        action1p1=decision(statep1,["bet","raise","fold"])
                        Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
                        actionp1 = action1p1
                        if (actionp1 == "bet"):
                            pot=pot+betsize
                            rewardp1 = -betsize
                            betsize = 1
                            break
                        elif (actionp1=='raise'):
                            betsize = betsize+1
                            pot = pot+betsize
                            rewardp1=-betsize
                        elif (actionp1== 'fold'):
                            Q = updateQ(Q,statep1,None,actionp1,None,0,gamma)
                            Q = updateQ(Q,statep2,None,actionp2,None,pot+0,gamma)
                            return Q
                    elif (actionp2== 'fold'):
                        Q = updateQ(Q,statep2,None,actionp2,None,0,gamma)
                        Q = updateQ(Q,statep1,None,actionp1,None,pot+0,gamma)
                        return Q
            elif (actionp1 == "fold"):
                Q = updateQ(Q,statep1,None,"fold",None,rewardp1,gamma)
                Q = updateQ(Q,statep2,None,"bet",None,pot+rewardp2,gamma)
                return Q
    #postflop statefinding
    board,deck=flop(deck)
    state1p1 = state(p1,board,"flop")
    state1p2 = state(p2,board,"flop")
    action1p1 = decision(state1p1,["bet","check"])
    Q = updateQ(Q,statep1,state1p1,actionp1,action1p1,rewardp1,gamma)
    statep1 = state1p1
    actionp1 = action1p1
    #postflop betting
    if (actionp1 == "bet"):
        pot = pot+betsize
        rewardp1 = -betsize
        action1p2 = decision(state1p2,["bet","fold","raise"])
        Q = updateQ(Q,statep2,state1p2,actionp2,action1p2,rewardp2,gamma)
        statep2 = state1p2
        actionp2=action1p2
        if (actionp2 == "bet"):
            pot = pot+betsize
            rewardp2 = -betsize
        elif (actionp2 == "raise"):
            betsize = betsize + 1
            pot = pot+betsize
            rewardp2 = -betsize-ante
            while (betsize <5):
                action1p1 = decision(statep1,["bet","raise","fold"])
                Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
                actionp1 = action1p1
                if (actionp1 == "bet"):
                    pot=pot+betsize
                    rewardp1 = -betsize
                    betsize = 1
                    break
                elif (actionp1 == "raise"):
                    betsize = betsize+1
                    pot = pot+betsize
                    rewardp1 = -betsize
                    action1p2=decision(statep2,["bet","raise","fold"])
                    Q = updateQ(Q,statep2,statep2,actionp2,action1p2,rewardp2,gamma)
                    actionp2 = action1p2
                    if (actionp2 == "bet"):
                        pot=pot+betsize
                        rewardp2 = -betsize
                        betsize = 1
                        break
                    elif (actionp2=='raise'):
                        betsize = betsize+1
                        pot = pot+betsize
                        rewardp2=-betsize
                    elif (actionp2 == 'fold'):
                        Q = updateQ(Q,statep2,None,actionp2,None,0,gamma)
                        Q = updateQ(Q,statep1,None,actionp1,None,pot+0,gamma)
                        return Q
                elif (actionp1 == 'fold'):
                    Q = updateQ(Q,statep1,None,actionp1,None,0,gamma)
                    Q = updateQ(Q,statep2,None,actionp2,None,pot+0,gamma)
                    return Q
        elif (actionp2 == "fold"):
            Q = updateQ(Q,statep2,None,"fold",None,0,gamma)
            Q = updateQ(Q,statep1,None,"bet",None,pot+0,gamma)
            return Q
    elif (actionp1 == "check"):
        rewardp1 = 0
        action1p2 = decision(state1p2,["bet","check"])
        Q = updateQ(Q,statep2,state1p2,actionp2,action1p2,rewardp2,gamma)
        statep2 = state1p2
        actionp2=action1p2
        if (actionp2 == "check"):
            rewardp2 = 0
        elif (actionp2 == "bet"):
            pot = pot+betsize
            rewardp2=-betsize
            action1p1=decision(statep1,["bet","fold","raise"])
            Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
            statep1 = state1p1
            actionp1 = action1p1
            if (actionp1 == "bet"):
                pot=pot+betsize
                rewardp1 = -betsize
            elif (actionp1 == "raise"):
                betsize = betsize + 1
                pot = pot+betsize
                rewardp1 = -betsize-ante
                while (betsize <5):
                    action1p2 = decision(statep2,["bet","raise","fold"])
                    Q = updateQ(Q,statep2,statep2,actionp2,action1p2,rewardp2,gamma)
                    actionp2 = action1p2
                    if (actionp2 == "bet"):
                        pot=pot+betsize
                        rewardp2 = -betsize
                        betsize = 1
                        break
                    elif (actionp2 == "raise"):
                        betsize = betsize+1
                        pot = pot+betsize
                        rewardp2 = -betsize
                        action1p1=decision(statep1,["bet","raise","fold"])
                        Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
                        actionp1 = action1p1
                        if (actionp1 == "bet"):
                            pot=pot+betsize
                            rewardp1 = -betsize
                            betsize = 1
                            break
                        elif (actionp1=='raise'):
                            betsize = betsize+1
                            pot = pot+betsize
                            rewardp1=-betsize
                        elif (actionp1== 'fold'):
                            Q = updateQ(Q,statep1,None,actionp1,None,0,gamma)
                            Q = updateQ(Q,statep2,None,actionp2,None,pot+0,gamma)
                            return Q
                    elif (actionp2== 'fold'):
                        Q = updateQ(Q,statep2,None,actionp2,None,0,gamma)
                        Q = updateQ(Q,statep1,None,actionp1,None,pot+0,gamma)
                        return Q
            elif (actionp1 == "fold"):
                Q = updateQ(Q,statep1,None,"fold",None,0,gamma)
                Q = updateQ(Q,statep2,None,"bet",None,pot+0,gamma)
                return Q   
    #postturn statefinding and Q updating
    board,deck=turn(board,deck)
    state1p1 = state(p1,board,"turn")
    state1p2 = state(p2,board,"turn")
    action1p1 = decision(state1p1,["bet","check"])
    Q = updateQ(Q,statep1,state1p1,actionp1,action1p1,rewardp1,gamma)
    statep1 = state1p1
    actionp1 = action1p1
    #postturn betting
    if (actionp1 == "bet"):
        pot = pot+betsize
        rewardp1 = -betsize
        action1p2 = decision(state1p2,["bet","fold","raise"])
        Q = updateQ(Q,statep2,state1p2,actionp2,action1p2,rewardp2,gamma)
        statep2 = state1p2
        actionp2=action1p2
        if (actionp2 == "bet"):
            pot = pot+betsize
            rewardp2 = -betsize
        elif (actionp2 == "raise"):
            betsize = betsize + 1
            pot = pot+betsize
            rewardp2 = -betsize-ante
            while (betsize <5):
                action1p1 = decision(statep1,["bet","raise","fold"])
                Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
                actionp1 = action1p1
                if (actionp1 == "bet"):
                    pot=pot+betsize
                    rewardp1 = -betsize
                    betsize = 1
                    break
                elif (actionp1 == "raise"):
                    betsize = betsize+1
                    pot = pot+betsize
                    rewardp1 = -betsize
                    action1p2=decision(statep2,["bet","raise","fold"])
                    Q = updateQ(Q,statep2,statep2,actionp2,action1p2,rewardp2,gamma)
                    actionp2 = action1p2
                    if (actionp2 == "bet"):
                        pot=pot+betsize
                        rewardp1 = -betsize
                        betsize = 1
                        break
                    elif (actionp2=='raise'):
                        betsize = betsize+1
                        pot = pot+betsize
                        rewardp2=-betsize
                    elif (actionp2 == 'fold'):
                        Q = updateQ(Q,statep2,None,actionp2,None,0,gamma)
                        Q = updateQ(Q,statep1,None,actionp1,None,pot+0,gamma)
                        return Q
                elif (actionp1 == 'fold'):
                    Q = updateQ(Q,statep1,None,actionp1,None,0,gamma)
                    Q = updateQ(Q,statep2,None,actionp2,None,pot+0,gamma)
                    return Q
        elif (actionp2 == "fold"):
            Q = updateQ(Q,statep2,None,"fold",None,0,gamma)
            Q = updateQ(Q,statep1,None,"bet",None,pot+0,gamma)
            return Q
    elif (actionp1 == "check"):
        rewardp1 = 0
        action1p2 = decision(state1p2,["bet","check"])
        Q = updateQ(Q,statep2,state1p2,actionp2,action1p2,rewardp2,gamma)
        statep2 = state1p2
        actionp2=action1p2
        if (actionp2 == "check"):
            rewardp2 = 0
        elif (actionp2 == "bet"):
            pot = pot+betsize
            rewardp2=-betsize
            action1p1=decision(statep1,["bet","fold","raise"])
            Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
            statep1 = state1p1
            actionp1 = action1p1
            if (actionp1 == "bet"):
                pot=pot+betsize
                rewardp1 = -betsize
            elif (actionp1 == "raise"):
                betsize = betsize + 1
                pot = pot+betsize
                rewardp1 = -betsize-ante
                while (betsize <5):
                    action1p2 = decision(statep2,["bet","raise","fold"])
                    Q = updateQ(Q,statep2,statep2,actionp2,action1p2,rewardp2,gamma)
                    actionp2 = action1p2
                    if (actionp2 == "bet"):
                        pot=pot+betsize
                        rewardp2 = -betsize
                        betsize = 1
                        break
                    elif (actionp2 == "raise"):
                        betsize = betsize+1
                        pot = pot+betsize
                        rewardp2 = -betsize
                        action1p1=decision(statep1,["bet","raise","fold"])
                        Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
                        actionp1 = action1p1
                        if (actionp1 == "bet"):
                            pot=pot+betsize
                            rewardp1 = -betsize
                            betsize = 1
                            break
                        elif (actionp1=='raise'):
                            betsize = betsize+1
                            pot = pot+betsize
                            rewardp1=-betsize
                        elif (actionp1== 'fold'):
                            Q = updateQ(Q,statep1,None,actionp1,None,0,gamma)
                            Q = updateQ(Q,statep2,None,actionp2,None,pot+0,gamma)
                            return Q
                    elif (actionp2== 'fold'):
                        Q = updateQ(Q,statep2,None,actionp2,None,0,gamma)
                        Q = updateQ(Q,statep1,None,actionp1,None,pot+0,gamma)
                        return Q
            elif (actionp1 == "fold"):
                Q = updateQ(Q,statep1,None,"fold",None,0,gamma)
                Q = updateQ(Q,statep2,None,"bet",None,pot+0,gamma)
                return Q   
    #postriver statefinding and Q updating
    board,deck=river(board,deck)
    state1p1 = state(p1,board,"river")
    state1p2 = state(p2,board,"river")
    action1p1 = decision(state1p1,["bet","check"])
    Q = updateQ(Q,statep1,state1p1,actionp1,action1p1,rewardp1,gamma)
    statep1 = state1p1
    actionp1 = action1p1
    #postriver betting
    if (actionp1 == "bet"):
        pot = pot+betsize
        rewardp1 = -betsize
        action1p2 = decision(state1p2,["bet","fold","raise"])
        Q = updateQ(Q,statep2,state1p2,actionp2,action1p2,rewardp2,gamma)
        statep2 = state1p2
        actionp2=action1p2
        if (actionp2 == "bet"):
            pot = pot+betsize
            rewardp2 = -betsize
        elif (actionp2 == "raise"):
            betsize = betsize + 1
            pot = pot+betsize
            rewardp2 = -betsize-ante
            while (betsize <5):
                action1p1 = decision(statep1,["bet","raise","fold"])
                Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
                actionp1 = action1p1
                if (actionp1 == "bet"):
                    pot=pot+betsize
                    rewardp1 = -betsize
                    betsize = 1
                    break
                elif (actionp1 == "raise"):
                    betsize = betsize+1
                    pot = pot+betsize
                    rewardp1 = -betsize
                    action1p2=decision(statep2,["bet","raise","fold"])
                    Q = updateQ(Q,statep2,statep2,actionp2,action1p2,rewardp2,gamma)
                    actionp2 = action1p2
                    if (actionp2 == "bet"):
                        pot=pot+betsize
                        rewardp1 = -betsize
                        betsize = 1
                        break
                    elif (actionp2=='raise'):
                        betsize = betsize+1
                        pot = pot+betsize
                        rewardp2=-betsize
                    elif (actionp2 == 'fold'):
                        Q = updateQ(Q,statep2,None,actionp2,None,0,gamma)
                        Q = updateQ(Q,statep1,None,actionp1,None,pot+0,gamma)
                        return Q
                elif (actionp1 == 'fold'):
                    Q = updateQ(Q,statep1,None,actionp1,None,0,gamma)
                    Q = updateQ(Q,statep2,None,actionp2,None,pot+0,gamma)
                    return Q
        elif (actionp2 == "fold"):
            Q = updateQ(Q,statep2,None,"fold",None,0,gamma)
            Q = updateQ(Q,statep1,None,"bet",None,pot+0,gamma)
            return Q
    elif (actionp1 == "check"):
        rewardp1 = 0
        action1p2 = decision(state1p2,["bet","check"])
        Q = updateQ(Q,statep2,state1p2,actionp2,action1p2,rewardp2,gamma)
        statep2 = state1p2
        actionp2=action1p2
        if (actionp2 == "check"):
            rewardp2 = 0
        elif (actionp2 == "bet"):
            pot = pot+betsize
            rewardp2=-betsize
            action1p1=decision(statep1,["bet","fold","raise"])
            Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
            statep1 = state1p1
            actionp1 = action1p1
            if (actionp1 == "bet"):
                pot=pot+betsize
                rewardp1 = -betsize
            elif (actionp1 == "raise"):
                betsize = betsize + 1
                pot = pot+betsize
                rewardp1 = -betsize-ante
                while (betsize <5):
                    action1p2 = decision(statep2,["bet","raise","fold"])
                    Q = updateQ(Q,statep2,statep2,actionp2,action1p2,rewardp2,gamma)
                    actionp2 = action1p2
                    if (actionp2 == "bet"):
                        pot=pot+betsize
                        rewardp2 = -betsize
                        betsize = 1
                        break
                    elif (actionp2 == "raise"):
                        betsize = betsize+1
                        pot = pot+betsize
                        rewardp2 = -betsize
                        action1p1=decision(statep1,["bet","raise","fold"])
                        Q = updateQ(Q,statep1,statep1,actionp1,action1p1,rewardp1,gamma)
                        actionp1 = action1p1
                        if (actionp1 == "bet"):
                            pot=pot+betsize
                            rewardp1 = -betsize
                            betsize = 1
                            break
                        elif (actionp1=='raise'):
                            betsize = betsize+1
                            pot = pot+betsize
                            rewardp1=-betsize
                        elif (actionp1== 'fold'):
                            Q = updateQ(Q,statep1,None,actionp1,None,0,gamma)
                            Q = updateQ(Q,statep2,None,actionp2,None,pot+0,gamma)
                            return Q
                    elif (actionp2== 'fold'):
                        Q = updateQ(Q,statep2,None,actionp2,None,0,gamma)
                        Q = updateQ(Q,statep1,None,actionp1,None,pot+0,gamma)
                        return Q
            elif (actionp1 == "fold"):
                Q = updateQ(Q,statep1,None,"fold",None,0,gamma)
                Q = updateQ(Q,statep2,None,"bet",None,pot+0,gamma)
                return Q    
    #find winner and final Q updates
    if (find_winner(p1,p2,board)=='p1'):
        Q = updateQ(Q,statep1,None,actionp1,None,pot+rewardp1,gamma)
        Q = updateQ(Q,statep2,None,actionp2,None,rewardp2,gamma)
    elif (find_winner(p1,p2,board)=='p2'):
        Q = updateQ(Q,statep2,None,actionp2,None,pot+rewardp2,gamma)
        Q = updateQ(Q,statep1,None,actionp1,None,rewardp1,gamma)
    elif (find_winner(p1,p2,board)=='split'):
        Q = updateQ(Q,statep2,None,actionp2,None,pot/2+rewardp2,gamma)
        Q = updateQ(Q,statep1,None,actionp1,None,pot/2+rewardp1,gamma)
    return Q
    
    
    
    

In [17]:
def iterator(Q,gamma,iterations):
    for i in range(1,iterations+1):
        Q=simgame(Q,gamma)
    return Q

In [31]:
start = time.time()

Q = iterator(Q,.05,10000)

end = time.time()
print("Runtime:", end - start)

Runtime: 354.4536838531494


## Display Q Table

In [32]:
Q

,bet,raise,check,fold,state
0,-0.018601,-0.347376,0.267100,0.000000,"(0, 0, 0, 0, flop)"
1,-0.077977,-1.294997,0.166938,0.000000,"(0, 0, 0, 0, turn)"
2,-0.012200,-1.178281,0.194891,0.000000,"(0, 0, 0, 0, river)"
3,0.042296,-0.333610,0.393677,0.000000,"(0, 0, 1, 0, flop)"
4,-0.047743,-1.087964,0.468899,0.000000,"(0, 0, 1, 0, turn)"
...,...,...,...,...,...
717,-0.049091,-0.077864,0.415476,-0.024375,"(12, 12, False)"
718,-0.046875,0.062500,0.167905,-0.012500,"(12, 13, True)"
719,-0.341988,-0.307500,0.194286,-0.213781,"(12, 13, False)"
720,-0.115587,0.000000,0.304792,-0.012500,"(13, 13, True)"


In [27]:
Q.to_csv(r"C:\Users\cnaka\Class Notes\32020 Reinforcement Learning\Q.csv")